# Pipeline test notebook

### Import requirements

In [73]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import time

from ImageProcessor import ImageProcessor
from StereoMatcher import StereoMatcher

%matplotlib ipympl

### Load calibrations and other data

In [74]:
imageProcessor = ImageProcessor()
imageProcessor.loadMonoCalibrationResults()
imageProcessor.loadStereoCalibration()
imageProcessor.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [75]:
stereoMatcher = StereoMatcher("SGBM", \
                vertical=True, createRightMatcher=False)
stereoMatcher.referenceImageProcessor(imageProcessor)

imageProcessor.initUndistortRectifyMap()
#stereoMatcher.createDisparityWLSFilter()

Reading from data/parametersSGBM.json


### Loading images

In [76]:
path = "captures/testImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))
print ("Selections: 0-{}".format(len(imageGlobL)-1))

Selections: 0-7


### Select image pair and display

In [95]:
imageNumber = 6

imageL = cv2.imread(imageGlobL[imageNumber])
imageR = cv2.imread(imageGlobR[imageNumber])

plt.figure()
plt.imshow(cv2.cvtColor(np.hstack([imageL, imageL]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Convert to grayscale and undistort

In [96]:
imageProcessor.convertToGrayscale(imageL, imageR)
imageProcessor.undistortRectifyRemap(imageProcessor.grayImageL, \
                                        imageProcessor.grayImageR)

### View undistorted image

In [97]:
fig = plt.figure()
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(np.hstack([imageProcessor.undistortImageL, \
            imageProcessor.undistortImageR]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
fig = plt.figure()
fig.suptitle("horizontal epipolar")
plt.imshow(cv2.cvtColor(imageProcessor.drawHorEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [122]:
fig = plt.figure(figsize=(6,15))
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(imageProcessor.drawVertEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute disparity map

In [100]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessor.undistortImageL, \
                grayImageR=imageProcessor.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()
#stereoMatcher.applyWLSFilterDisparity()

print ("minDisparity:", stereoMatcher.disparityMapL.min())
print ("maxDisparity:", stereoMatcher.disparityMapL.max())

minDisparity: 9.0
maxDisparity: 37.625


### View disparity map

In [101]:
plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE), cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map

In [102]:
focalLength = imageProcessor.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = np.empty_like(stereoMatcher.disparityMapL)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

9.0
37.625
(640, 360)
630.07556
2634.066


### View depth map

In [103]:
plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE), cmap="Greys")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute point cloud

In [104]:
points = cv2.reprojectImageTo3D(stereoMatcher.disparityMapL, \
                                imageProcessor.dispToDepthMatrix)

# Reshaping to a list of 3D coordinates
pointCloud = points.reshape((points.shape[0]*points.shape[1],3))[0::16]\
                                                        .astype(np.int16)

print (points.shape)
print (pointCloud.shape)

print (points[0,0])
print (pointCloud[0])

(640, 360, 3)
(14400, 3)
[ -713.31146 -1168.7544   2763.0342 ]
[ -713 -1168  2763]


Filtering extreme points

In [105]:
# Filtering x values
pointCloud = pointCloud[np.logical_and(\
    pointCloud[:, 0]>pointCloud[:, 0].min(), \
    pointCloud[:, 0]<pointCloud[:, 0].max())]
# Filtering y values
pointCloud = pointCloud[np.logical_and(\
    pointCloud[:, 1]>pointCloud[:, 1].min(), \
    pointCloud[:, 1]<pointCloud[:, 1].max())]
# Filtering z values
pointCloud = pointCloud[np.logical_and(\
    pointCloud[:, 2]>pointCloud[:, 2].min(), \
    pointCloud[:, 2]<pointCloud[:, 2].max())]

print (pointCloud.shape)

(11378, 3)


Rotate point cloud to have y forward

In [106]:
pointRotMatrix = np.array([ [ 0,  0, -1],
                            [-1,  0,  0],
                            [ 0,  1,  0] ])

pointCloud = np.dot(pointCloud[:], pointRotMatrix)

print (pointCloud.shape)
print (pointCloud[0])

(11378, 3)
[ 876 2072  355]


### View point cloud

In [107]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(pointCloud[:,0], pointCloud[:,1], pointCloud[:,2], s=1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_ylim(0,)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Voxelize point cloud

In [108]:
voxelSize = 10 # mm
voxelRange = np.array(
  [[pointCloud[:, 0].min(), pointCloud[:, 1].min(), pointCloud[:, 2].min()],\
  [pointCloud[:, 0].max(), pointCloud[:, 1].max(), pointCloud[:, 2].max()]])\
  .astype(np.int16)

print(voxelRange, "range")

voxelUnits = (((voxelRange[1]-voxelRange[0]) - \
            (voxelRange[1]-voxelRange[0]) % voxelSize)/10).astype(np.int16)
print(voxelUnits, "units")

voxelCenter = (voxelRange[0] + (voxelRange[1]-voxelRange[0])/2 - (voxelRange[0] + (voxelRange[1]-voxelRange[0])/2)%10).astype(np.int16)
print(voxelCenter, "center")

[[-816  673 -250]
 [1048 2486  480]] range
[186 181  73] units
[ 110 1570  110] center


In [109]:
# Sampling
voxelSize = 50 # mm
iterations = 0
occupyThreshold = 5
occupancy = []

startTime = time.perf_counter()

while len(pointCloud>0):

    sampledPoint = pointCloud[0]

    samplingLimit = np.empty_like(sampledPoint)
    for n in range(len(sampledPoint)):
        samplingLimit[n]=(sampledPoint[n]//voxelSize)*voxelSize

    mask = np.ones(pointCloud.shape[0], dtype=bool)

    for n in range(len(sampledPoint)):
        maskLower = pointCloud[:,n]>=samplingLimit[n]
        maskUpper = pointCloud[:,n]<samplingLimit[n]+voxelSize
        mask = np.logical_and(mask, np.logical_and(maskLower, maskUpper))
    pointsInVoxel = pointCloud[mask]

    if len(pointsInVoxel)>occupyThreshold:
        voxelMidpoint = samplingLimit+voxelSize/2
        occupancy.append(voxelMidpoint)

    pointCloud = pointCloud[np.invert(mask)]

    iterations+=1

occupancy = np.array(occupancy, dtype=np.int16)
print("Time taken: {:.5f}".format(time.perf_counter()-startTime))

Time taken: 2.47175


In [111]:
print("Iterations:", iterations)
print(occupancy)
print("Voxels:", occupancy.shape[0])

Iterations: 1706
[[ 675 1575   25]
 [ 425  975  -75]
 [ 425  975 -125]
 ...
 [-425  975   25]
 [-425  925   25]
 [-375  825 -175]]
Voxels: 480


### View voxelized point cloud

In [112]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(occupancy[:,0], occupancy[:,1], occupancy[:,2])
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_ylim(0,)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …